In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from datetime import datetime


import sys
sys.path.append("/home/kieran/code/Ant-mel/legendary_game_recs/")

#from legendary_game_recs.preprocessing import preprocess_1_cleaning

from preprocessing.preprocess import *

In [2]:
def drop_duplicates(df):
    df.sort_values('title')
    return df.drop_duplicates()
    

def delete_no_release_date(df):
    return df[df['release_date'] != "null"]

def cleaning_in_notebook(df):
    numeric_columns = ['plays','playing','backlogs','wishlist','total_reviews','total_lists']
    string_columns = ['developers','genres','platforms']
    df = date_reformatted(df, 'release_date')
    df[string_columns] = make_list_columns_to_lists(df, string_columns)
    for x in numeric_columns:
        numeric_objects_reformatted(df, x)
    df = delete_no_release_date(df)
    df['plays'] = np.where(df['plays'] < 0, 0, df['plays'])
    return df
    

In [3]:
df = pd.read_csv("../raw_data/all_game_data_v1.csv", low_memory=False)

df.head()

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,ratings_one_five,ratings_two_zero,ratings_two_five,ratings_three_zero,ratings_three_five,ratings_four_zero,ratings_four_five,ratings_five_zero,image,url
0,PilotXross,2030-12-20,1,0,2,2,['PilotXross'],0.0,"['Indie', 'Racing', 'Sport']",['SteamVR'],...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/pilotxross/
1,Plague Hunter,2029-02-07,0,0,2,7,[],0.0,"['Adventure', 'Indie']",['Windows PC'],...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/plague-hunter/
2,The God Slayer,2027-12-31,0,0,1,8,['Pathea Games'],0.0,['RPG'],['PlayStation 5'],...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/the-god-slayer/
3,Tales of the Death,2025-12-31,0,0,0,4,"['8 Million Bit', 'Hanoi Studios']",0.0,[],['Windows PC'],...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/tales-of-the-d...
4,Riversiders,2025-12-31,1,0,0,0,[],0.0,"['Adventure', 'Racing']",['Windows PC'],...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/riversiders/


In [4]:
df = drop_duplicates(df)

In [5]:
df

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,ratings_one_five,ratings_two_zero,ratings_two_five,ratings_three_zero,ratings_three_five,ratings_four_zero,ratings_four_five,ratings_five_zero,image,url
0,PilotXross,2030-12-20,1,0,2,2,['PilotXross'],0.0,"['Indie', 'Racing', 'Sport']",['SteamVR'],...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/pilotxross/
1,Plague Hunter,2029-02-07,0,0,2,7,[],0.0,"['Adventure', 'Indie']",['Windows PC'],...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/plague-hunter/
2,The God Slayer,2027-12-31,0,0,1,8,['Pathea Games'],0.0,['RPG'],['PlayStation 5'],...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/the-god-slayer/
3,Tales of the Death,2025-12-31,0,0,0,4,"['8 Million Bit', 'Hanoi Studios']",0.0,[],['Windows PC'],...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/tales-of-the-d...
4,Riversiders,2025-12-31,1,0,0,0,[],0.0,"['Adventure', 'Racing']",['Windows PC'],...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/riversiders/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109807,Taichi Panda,0001-01-01,0,0,0,0,[],0.0,[],[],...,0,0,0,0,0,0,0,0,NaN,https://www.backloggd.com/games/taichi-panda--1/
109808,Silverio Ragnarok,0001-01-01,1,0,2,3,[],0.0,[],[],...,0,0,0,0,1,0,0,0,NaN,https://www.backloggd.com/games/silverio-ragna...
109809,GhostSlayer,0001-01-01,1,0,1,0,[],0.0,[],[],...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/ghostslayer/
109810,Elifoot,0001-01-01,1,0,0,1,[],0.0,[],[],...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/elifoot/


In [6]:
df['release_date'] = df['release_date'].apply(remove_hyphens)

In [7]:
df['release_date'] = df['release_date'].apply(change_to_datetype)

In [8]:
df

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,ratings_one_five,ratings_two_zero,ratings_two_five,ratings_three_zero,ratings_three_five,ratings_four_zero,ratings_four_five,ratings_five_zero,image,url
0,PilotXross,12/20/2030,1,0,2,2,['PilotXross'],0.0,"['Indie', 'Racing', 'Sport']",['SteamVR'],...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/pilotxross/
1,Plague Hunter,02/07/2029,0,0,2,7,[],0.0,"['Adventure', 'Indie']",['Windows PC'],...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/plague-hunter/
2,The God Slayer,12/31/2027,0,0,1,8,['Pathea Games'],0.0,['RPG'],['PlayStation 5'],...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/the-god-slayer/
3,Tales of the Death,12/31/2025,0,0,0,4,"['8 Million Bit', 'Hanoi Studios']",0.0,[],['Windows PC'],...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/tales-of-the-d...
4,Riversiders,12/31/2025,1,0,0,0,[],0.0,"['Adventure', 'Racing']",['Windows PC'],...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/riversiders/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109807,Taichi Panda,null,0,0,0,0,[],0.0,[],[],...,0,0,0,0,0,0,0,0,NaN,https://www.backloggd.com/games/taichi-panda--1/
109808,Silverio Ragnarok,null,1,0,2,3,[],0.0,[],[],...,0,0,0,0,1,0,0,0,NaN,https://www.backloggd.com/games/silverio-ragna...
109809,GhostSlayer,null,1,0,1,0,[],0.0,[],[],...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/ghostslayer/
109810,Elifoot,null,1,0,0,1,[],0.0,[],[],...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/elifoot/


In [ ]:
df = cleaning_in_notebook(df)

In [ ]:
df.sort_values('release_date')

In [ ]:
df[df['plays'] == -1]

In [ ]:
df[df['title'] == 'Mytran Wars']

In [ ]:
df.sort_values('plays', ascending=False).head(20)

In [ ]:
df[['title','total_reviews','total_lists']].sort_values('total_reviews', ascending=False).head(20)

In [ ]:
df.info()

In [ ]:
df['release_date'] = df['release_date'].apply(change_to_datetype)

In [ ]:
import seaborn as sns

sns.histplot(data = df, x = 'plays')

In [ ]:
sns.boxplot(data = df, x = 'avg_review')

In [ ]:
df.value_counts("total_reviews")